In [113]:
# Kontrolle mit WErte auf 100 -> sollte mit reingebaut werden
# Prozentzahl in die Mitte von paygap


In [114]:
import copy

In [115]:
number_players = 100
init_green = 80
init_yellow = 20
goal_green = 20
goal_yellow = 80
earnings_min = 1000
earnings_max = 10000
years_develop = 30
years_displayed = 40
years_displayed_cumulatively = 5

In [116]:
temp_green = init_green
temp_yellow = init_yellow
move_green = (init_green-goal_green)*(-1)    #-60
move_yellow= (init_yellow-goal_yellow)*(-1)  #+60
ratio_development = []
ratio_development.append([temp_green, temp_yellow])
for i in range(round(years_develop/years_displayed_cumulatively)):
    temp_green = temp_green + (move_green/round(years_develop/years_displayed_cumulatively)) 
    temp_yellow = temp_yellow + (move_yellow/round(years_develop/years_displayed_cumulatively))
    ratio_development.append([round(temp_green), round(temp_yellow)])
if years_develop < years_displayed:
    d_years = years_displayed - years_develop
    for i in range(round(d_years/years_displayed_cumulatively)):
        ratio_development.append([round(temp_green), round(temp_yellow)])

print(ratio_development)

[[80, 20], [70, 30], [60, 40], [50, 50], [40, 60], [30, 70], [20, 80], [20, 80], [20, 80]]


In [117]:
d_earnings = earnings_max - earnings_min
d_earnings_pa = (d_earnings / (years_displayed/years_displayed_cumulatively)) # d_earnings_pa*Jahreszahl + earnings_min
temp_earnings = earnings_min
earnings_development=[]
for i in range(round(years_displayed/years_displayed_cumulatively)+1):
    earnings_development.append(round(temp_earnings))
    temp_earnings = temp_earnings + d_earnings_pa
earnings_development = earnings_development[::-1]
print(earnings_development)


[10000, 8875, 7750, 6625, 5500, 4375, 3250, 2125, 1000]


In [118]:
evolutions = []
init_evolution = []
for i in range(round(years_displayed/years_displayed_cumulatively)+1):
    init_evolution.append(ratio_development[0])
#evolutions.append(copy.deepcopy(init_evolution))
for p in range(len(ratio_development)):
    #p ist counter für überschreibende inhalte in der Liste
    init_evolution.append(copy.deepcopy(ratio_development[p]))
    del init_evolution[0]
    evolutions.append(copy.deepcopy(init_evolution))


In [119]:
print(evolutions)

[[[80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20]], [[80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [70, 30]], [[80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [70, 30], [60, 40]], [[80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [70, 30], [60, 40], [50, 50]], [[80, 20], [80, 20], [80, 20], [80, 20], [80, 20], [70, 30], [60, 40], [50, 50], [40, 60]], [[80, 20], [80, 20], [80, 20], [80, 20], [70, 30], [60, 40], [50, 50], [40, 60], [30, 70]], [[80, 20], [80, 20], [80, 20], [70, 30], [60, 40], [50, 50], [40, 60], [30, 70], [20, 80]], [[80, 20], [80, 20], [70, 30], [60, 40], [50, 50], [40, 60], [30, 70], [20, 80], [20, 80]], [[80, 20], [70, 30], [60, 40], [50, 50], [40, 60], [30, 70], [20, 80], [20, 80], [20, 80]]]


In [120]:
def earnings_per_evolution (earnings, data):
    sum_earnings_green = 0
    sum_earnings_yellow = 0
    sum_people_green = 0
    sum_people_yellow = 0
    count = 0
    for tup in data:
        sum_people_green += tup[0]
        sum_people_yellow += tup[1]
        sum_earnings_green += tup[0]*earnings[count]
        sum_earnings_yellow += tup[1]*earnings[count]
        count += 1
    return [sum_earnings_green/(sum_people_green),sum_earnings_yellow/(sum_people_yellow)]


In [121]:
sums =  []
for gen in evolutions:
    sums.append(earnings_per_evolution(earnings_development, gen))
print(sums)

[[5500.0, 5500.0], [5563.380281690141, 5263.1578947368425], [5679.347826086957, 4910.714285714285], [5840.909090909091, 4562.5], [6044.354838709677, 4294.642857142857], [6289.473684210527, 4136.363636363636], [6580.882352941177, 4086.5384615384614], [6800.0, 4200.0], [6913.461538461538, 4419.117647058823]]


In [122]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
import io

In [123]:
def create_frame(data_frame, left_labels=None, right_labels=None, center_left_label=None, center_right_label=None, width=400, height_per_bar=30):
    import matplotlib.pyplot as plt
    from matplotlib.patches import Rectangle
    import io
    from PIL import Image

    if left_labels is None:
        left_labels = list(range(len(data_frame)))
    if right_labels is None:
        right_labels = [""] * len(data_frame)
    if center_left_label is None:
        center_left_label = "" 
    if center_right_label is None:
        center_right_label = "" 

    total_height = height_per_bar * len(data_frame)
    fig, ax = plt.subplots(figsize=(width / 100, total_height / 100), dpi=100)

    # Set the axis limits
    ax.set_xlim(0, 100)
    ax.set_ylim(0, total_height)
    ax.yaxis.set_ticks([]) 
    ax.tick_params(left=False)

    # Draw the "Jahre" and "Gehalt" labels at the top
    ax.text(-5, total_height + 5, "Jahre", va='center', ha='right', fontsize=10, fontweight='bold')
    ax.text(105, total_height + 5, "Gehalt", va='center', ha='left', fontsize=10, fontweight='bold')

    # Draw the bars with their labels and values
    for i, ((green, yellow), left_label, right_label) in enumerate(zip(data_frame, left_labels, right_labels)):
        bottom = total_height - (i + 1) * height_per_bar

        ax.add_patch(Rectangle((0, bottom), green, height_per_bar, color='green'))
        ax.add_patch(Rectangle((green, bottom), yellow, height_per_bar, color='yellow'))
        
        ax.plot([0, 100], [bottom, bottom], color='black', linewidth=0.5)
        
        ax.text(-5, bottom + height_per_bar / 2, str(left_label), va='center', ha='right', fontsize=8)
        ax.text(105, bottom + height_per_bar / 2, str(right_label), va='center', ha='left', fontsize=8)


    ax.set_xticks([0, 20, 40, 60, 80, 100])
    ax.set_xticklabels(['0', '20', '40', '60', '80', '100'])
    ax.xaxis.set_ticks_position('bottom')
    ax.plot([0, 100], [0, 0], color='black', linewidth=0.5)


    # cumulated values
    ax.text(0, -total_height / 7, str(center_left_label) + " " + str(round((center_left_label*100)/(center_left_label+center_right_label))) + "%", va='center', ha='left', fontsize=10, fontweight='bold')
    ax.text(100, -total_height / 7, str(center_right_label) + " " + str(round((center_right_label*100)/(center_left_label+center_right_label))) + "%", va='center', ha='right', fontsize=10, fontweight='bold')


    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)


In [124]:
def create_gif(data_3d,  output_path="balken_animation.gif", duration=200):
    frames = [create_frame(data_3d[i], left_labels=list(range(years_displayed, -1, -1*years_displayed_cumulatively)), right_labels=earnings_development, center_left_label=round(sums[i][0]), center_right_label=round(sums[i][1])) for i in range(len(data_3d))]
    frames[0].save(output_path, format='GIF', append_images=frames[1:], save_all=True, duration=duration, loop=0)

In [125]:
create_gif(evolutions, "balkenverlauf.gif", duration=300)

In [126]:
import os
def save_frames_as_images(data_frames, left_labels=None, right_labels=None, center_labels=None, output_folder='frames', width=400, height_per_bar=30):
    """
    Speichert die Frames als Bilder in einem Ordner. `center_labels` ist eine Liste von Paaren,
    wobei jedes Paar die linken und rechten Labels für das jeweilige Bild enthält.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Durchlaufe alle Frames und speichere sie
    for idx, data_frame in enumerate(data_frames):
        # Extrahiere die entsprechenden center_labels für das aktuelle Bild
        center_left_label, center_right_label = center_labels[idx]
        
        # Erstelle das Bild und speichere es
        img = create_frame(data_frame, left_labels, right_labels, center_left_label, center_right_label, width, height_per_bar)
        img_path = os.path.join(output_folder, f"frame_{idx + 1}.png")
        img.save(img_path)

In [127]:
save_frames_as_images(evolutions, left_labels=list(range(years_displayed, -1, -1*years_displayed_cumulatively)), right_labels=earnings_development, center_labels=[[round(s[0]), round(s[1])] for s in sums])